In [1]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Import classes from Class.py
from Data_handling_Konsti import *
from Class_Konsti import *
#from Class_no_zone import *

In [2]:
## PARAMETERS DEFINITION

# Time
H = 24          # Hours in a day
D = 5           # Typical days in a year
Y = 30          # Years of the investment timeline
N = H*D*Y       # Number of hours in the investment timeline    
N_Scenarios = 10
epsilon= 0.1
# Number of loads and generators
N_dem = len(Dem[0,:])       # Number of loads
N_gen_E = len(Gen_E_OpCost)   # Number of existing generators
N_gen_N = len(Gen_N_OpCost)   # Number of new generators
N_zone = len(Trans_Z_Connected_To_Z)     # Number of zones
N_line = len(Trans_Line_From_Z)   # Number of transmission lines


# Hyperparameters
B = 1000000000   # Budget for the investment problem
R = 73 # Conversion rate


In [3]:
## CREATE THE PARAMETERS AND DATA OBJECTS
ParametersObj = Parameters(H, D, Y, N, N_dem, N_gen_E, N_gen_N, N_zone, N_line, B, R, N_Scenarios,max_deviation,epsilon)
DataObj = InputData(Dem, Uti, Load_Z, Gen_E_OpCost, Gen_N_OpCost, Gen_N_MaxInvCap, Gen_E_Cap, Gen_N_InvCost, Gen_E_Tech, Gen_N_Tech, Gen_E_Z, Gen_N_Z, Gen_E_OpCap, Gen_N_OpCap, Trans_React, Trans_Cap, Trans_Line_From_Z, Trans_Line_To_Z, Trans_Z_Connected_To_Z,scenarios)

# Model 1: Sequential optimization of Dispatch problem and Investment problem

### 1) Market Clearing

In [4]:
# Run the Market Clearing Problem
MarketClearing1 = MarketClearingModel1(ParametersObj, DataObj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-29
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 140400 rows, 126000 columns and 270000 nonzeros
Model fingerprint: 0x4c1471a4
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+01, 1e+03]
  Bounds range     [1e+03, 1e+03]
  RHS range        [4e-01, 1e+03]
Presolve removed 134239 rows and 109069 columns
Presolve time: 0.57s
Presolved: 6161 rows, 16931 columns, 19670 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.739e+03
 Factor NZ  : 8.900e+03 (roughly 10 MB of memory)
 Factor Ops : 1.438e+04 (less than 1 second per iteration)
 Threads   

In [5]:
#MarketClearing1.res.df.head(10)

In [5]:
class InvestmentModel_Stochastic():
    def __init__(self, Parameters, Data, DA_Price, Model_results = 1, Guroby_results = 1):
        self.D = Data  # Data
        self.P = Parameters  # Parameters
        self.DA_Price = DA_Price  # Day-ahead price
        self.Model_results = Model_results  # Display results
        self.Guroby_results = Guroby_results  # Display guroby results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        self.var.P_N = self.m.addMVar((self.P.N_gen_N), lb=0) # Invested capacity in every new generator
        self.var.p_N = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0) # Power output per hour for every new generator


    def _build_constraints(self):
        # Capacity investment constraint
        self.con.cap_inv = self.m.addConstr(self.var.P_N <= self.D.Gen_N_MaxInvCap, name='Maximum capacity investment')

        # Max production constraint
        self.con.max_p_N = self.m.addConstr(self.var.p_N <= self.D.Gen_N_OpCap * self.var.P_N, name='Maximum RES production') 

        # Budget constraint
        for s in range(self.P.N_S):
            self.con.budget = self.m.addConstr(gp.quicksum(self.var.P_N [g] * self.D.scenarios[g,s] for g in range(self.P.N_gen_N)) <= self.P.B, name='Budget constraint')


    def _build_objective(self):
        revenues = ((self.var.p_N @ self.D.Gen_N_Z.T) * self.DA_Price).sum()  # don't use quicksum here because it's a <MLinExpr (3600, N_zone)>
        op_costs = gp.quicksum(self.var.p_N @ self.D.Gen_N_OpCost)
        invest_costs = gp.quicksum((1/self.P.N_S)*self.var.P_N[g] * self.D.scenarios[g,s] for g in range(self.P.N_gen_N) for s in range(self.P.N_S))
        objective = self.P.R*(revenues - op_costs) - invest_costs
        self.m.setObjective(objective, GRB.MAXIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)
    

    def _build_model(self):
        self.m = gp.Model('Investment problem')
        self._build_variables()  
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.optimize()
        if self.Model_results == 1:
            self._extract_results()

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        
        # Display the generators the model invested in, in a dataframe
        self.res.P_N = self.var.P_N.X
        self.res.P_N = self.res.P_N.reshape((self.P.N_gen_N,1))
        self.res.df = pd.DataFrame(self.D.Gen_N_Tech, columns = ['Technology'])
        self.res.df['Invested capacity (MW)'] = self.res.P_N       

In [7]:
class InvestmentModel_ChanceConstraint():
    def __init__(self, Parameters, Data, DA_Price, Model_results = 1, Guroby_results = 1):
        self.D = Data  # Data
        self.P = Parameters  # Parameters
        self.DA_Price = DA_Price  # Day-ahead price
        self.Model_results = Model_results  # Display results
        self.Guroby_results = Guroby_results  # Display guroby results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        self.var.P_N = self.m.addMVar((self.P.N_gen_N), lb=0) # Invested capacity in every new generator
        self.var.p_N = self.m.addMVar((self.P.N, self.P.N_gen_N), lb=0) # Power output per hour for every new generator
        self.var.u = self.m.addMVar((self.P.N_S), vtype=GRB.BINARY) # Binary variable for each scenario


    def _build_constraints(self):
        # Capacity investment constraint
        self.con.cap_inv = self.m.addConstr(self.var.P_N <= self.D.Gen_N_MaxInvCap, name='Maximum capacity investment')

        # Max production constraint
        self.con.max_p_N = self.m.addConstr(self.var.p_N <= self.D.Gen_N_OpCap * self.var.P_N, name='Maximum RES production') 

        # Budget constraint
        for s in range(self.P.N_S):
            self.con.budget = self.m.addConstr(gp.quicksum(self.var.P_N [g] * self.D.scenarios[g,s] for g in range(self.P.N_gen_N)) - self.P.B <= (1-self.var.u[s])*self.P.Big_M, name='Budget constraint')

        # Chance constraint
        self.con.chance = self.m.addConstr(gp.quicksum(self.var.u[s] for s in range(self.P.N_S))/self.P.N_S >= (1-self.P.epsilon), name='Chance constraint')


    def _build_objective(self):
        revenues = ((self.var.p_N @ self.D.Gen_N_Z.T) * self.DA_Price).sum()  # don't use quicksum here because it's a <MLinExpr (3600, N_zone)>
        op_costs = gp.quicksum(self.var.p_N @ self.D.Gen_N_OpCost)
        invest_costs = gp.quicksum((1/self.P.N_S)*self.var.P_N[g] * self.D.scenarios[g,s] for g in range(self.P.N_gen_N) for s in range(self.P.N_S))
        objective = self.P.R*(revenues - op_costs) - invest_costs
        self.m.setObjective(objective, GRB.MAXIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)
    

    def _build_model(self):
        self.m = gp.Model('Investment problem')
        self._build_variables()  
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.optimize()
        if self.Model_results == 1:
            self._extract_results()

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        #print values for binary variable
        print('Binary variable: ', self.var.u.X)
        # Display the generators the model invested in, in a dataframe
        self.res.P_N = self.var.P_N.X
        self.res.P_N = self.res.P_N.reshape((self.P.N_gen_N,1))
        self.res.df = pd.DataFrame(self.D.Gen_N_Tech, columns = ['Technology'])
        self.res.df['Invested capacity (MW)'] = self.res.P_N       

### 2) Investment Problem

In [8]:
#Run the investmentmodel
Investment_Stochastic = InvestmentModel_Stochastic(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 57866 rows, 57616 columns and 112076 nonzeros
Model fingerprint: 0xc07334db
Coefficient statistics:
  Matrix range     [4e-04, 2e+06]
  Objective range  [1e+03, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+09]
Presolve removed 57856 rows and 57600 columns
Presolve time: 0.08s
Presolved: 10 rows, 16 columns, 160 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9298568e+11   1.707565e+07   0.000000e+00      0s
       6    1.8839111e+10   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.11 seconds (0.05 work units)
Optimal objective  1.883911121e+10
Objective value:  18839111212.498608


In [9]:
Investment_Stochastic.res.df

,Technology,Invested capacity (MW)
0,Coal,0.000000
1,Coal,0.000000
2,Coal,106.336697
3,Gas,0.000000
4,Coal,0.000000
5,Wind,400.000000
6,Wind,15.473478
7,PV,0.000000
8,Gas,0.000000
9,Coal,0.000000


In [10]:
Investment_ChanceConstraint = InvestmentModel_ChanceConstraint(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 57867 rows, 57626 columns and 112096 nonzeros
Model fingerprint: 0xb1e6cdad
Variable types: 57616 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [4e-04, 2e+09]
  Objective range  [1e+03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 3e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -0.0000000
Presolve removed 57840 rows and 57584 columns
Presolve time: 0.10s
Presolved: 27 rows, 42 columns, 212 nonzeros
Variable types: 32 continuous, 10 integer (10 binary)

Root relaxation: objective 2.594366e+10, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node

In [11]:
Investment_ChanceConstraint.res.df

,Technology,Invested capacity (MW)
0,Coal,0.000000
1,Coal,0.000000
2,Coal,342.602707
3,Gas,0.000000
4,Coal,0.000000
5,Wind,400.000000
6,Wind,0.000000
7,PV,0.000000
8,Gas,0.000000
9,Coal,0.000000


In [12]:
# Export the results into the excel file called Investment Results and Market Clearing Results which are located in the folder Results
#InvestmentPB1.res.df.to_excel("../Results/InvestmentResults.xlsx")
#MarketClearing1.res.df.to_excel("../Results/MarketClearingResults.xlsx")

# Model 2: Integrated Bi-level optimization of dispatch problem and investment problem using KKTs